In [1]:
import re

import numpy as np
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import InfinityEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage

## 定数定義

In [2]:
EMBEDDING_MODEL_PATH = 'sbintuitions/sarashina-embedding-v2-1b'

## Embedding model

In [3]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


def show_similar_words(
    embed_model: InfinityEmbeddings,
    documents: list[str],
    query: str,
):
    doc_embeddings = embed_model.embed_documents(documents)
    query_embedding = np.array(embed_model.embed_query(query))

    # 類似度の計算
    similarities = []
    for i, doc_emb in enumerate(doc_embeddings):
        sim = cosine_similarity(query_embedding, doc_emb)
        similarities.append((i, sim, documents[i][:50] + "..."))

    # 類似度順にソート
    similarities.sort(key=lambda x: x[1], reverse=True)

    print(f'query: {query}')
    print("検索結果（類似度順）:")
    for rank, (idx, sim, text) in enumerate(similarities, 1):
        print(f"{rank}. 類似度: {sim:.4f} - {text}")

In [4]:
from openai import OpenAI
import json
client = OpenAI(
    base_url='http://proxy',
    api_key='EMPTY',
)

response = client.embeddings.create(
    input="task: クエリを与えるので、与えられたWeb検索クエリに答える関連文章を検索してください。\nquery: Sarashinaのテキスト埋め込みモデルはありますか?",
    model=EMBEDDING_MODEL_PATH
)
# json.loads(response.to_json())

In [5]:
embed = InfinityEmbeddings(
    model=EMBEDDING_MODEL_PATH,
    infinity_api_url='http://proxy',
)

### 単純なベクトル化

In [6]:
# 単純にベクトル化してみる
vector = np.array(embed.embed_query('task: クエリを与えるので、与えられたWeb検索クエリに答える関連文章を検索してください。\nquery: Sarashinaのテキスト埋め込みモデルはありますか?'))
vector

array([-0.00503895,  0.005185  , -0.00942064, ...,  0.01431353,
       -0.00119584,  0.01358325], shape=(1792,))

In [7]:
# 単純にベクトル化してみる
vector = np.array(embed.embed_query('私は、犬と遊ぶのが好きです。'))
vector.shape

(1792,)

### 少し難易度が高い場合

In [14]:
%%time
documents = [
    "text: 機械学習には、RandomForestやSVMなど様々なモデルがある",
    "text: CNNやViTは画像をクラス分類する技術です。",
    "text: 画像認識には、ResNetが有効",
    "text: yoloは物体検出を行うためのモデルである",
    "text: SwinTransformerは画像分類でかなり性能が高い",
    "text: 音声認識には、1DのCNNが有効",
    "text: 音声認識には、LSTMが有効",
    "text: データベースは情報を効率的に管理するシステムです。"
]
query = "task: クエリに関連した文章を検索してください \n query: 画像分類に有効なモデルは？"

show_similar_words(embed, documents, query)

query: task: クエリに関連した文章を検索してください 
 query: 画像分類に有効なモデルは？
検索結果（類似度順）:
1. 類似度: 0.8514 - text: 画像認識には、ResNetが有効...
2. 類似度: 0.8301 - text: CNNやViTは画像をクラス分類する技術です。...
3. 類似度: 0.8184 - text: SwinTransformerは画像分類でかなり性能が高い...
4. 類似度: 0.8156 - text: 機械学習には、RandomForestやSVMなど様々なモデルがある...
5. 類似度: 0.8106 - text: yoloは物体検出を行うためのモデルである...
6. 類似度: 0.8031 - text: 音声認識には、1DのCNNが有効...
7. 類似度: 0.7981 - text: 音声認識には、LSTMが有効...
8. 類似度: 0.7257 - text: データベースは情報を効率的に管理するシステムです。...
CPU times: user 7.39 ms, sys: 0 ns, total: 7.39 ms
Wall time: 225 ms


In [13]:
%%time
documents = [
    "text: トイプードルはしつけがしやすく、毛が抜けないのでアレルギーを持つ家族にもおすすめ",
    "text: 柴犬は可愛いが毛がよく抜けるのでアレルギーを持つ家族にはおすすめ出来ない",
    "text: ドーベルマンはしつけが難しい",
    "text: チワワは、小さくて可愛い",
    "text: ビットブルはとても危険",
    "text: ゴールデンレトリバーは温厚で飼いやすい",
    "text: ライオンを一般家庭で飼うのは難しい"
]
query = "task: クエリに関連した文章を検索してください \n query: アレルギーを持つ家族がいる場合におすすめのペットは？"

show_similar_words(embed, documents, query)

query: task: クエリに関連した文章を検索してください 
 query: アレルギーを持つ家族がいる場合におすすめのペットは？
検索結果（類似度順）:
1. 類似度: 0.8554 - text: トイプードルはしつけがしやすく、毛が抜けないのでアレルギーを持つ家族にもおすすめ...
2. 類似度: 0.8441 - text: 柴犬は可愛いが毛がよく抜けるのでアレルギーを持つ家族にはおすすめ出来ない...
3. 類似度: 0.7980 - text: ゴールデンレトリバーは温厚で飼いやすい...
4. 類似度: 0.7777 - text: ライオンを一般家庭で飼うのは難しい...
5. 類似度: 0.7646 - text: チワワは、小さくて可愛い...
6. 類似度: 0.7502 - text: ドーベルマンはしつけが難しい...
7. 類似度: 0.7116 - text: ビットブルはとても危険...
CPU times: user 3.75 ms, sys: 3.69 ms, total: 7.44 ms
Wall time: 299 ms


In [12]:
query = 'task: クエリを与えるので、与えられたWeb検索クエリに答える関連文章を検索してください。\nquery: Sarashinaのテキスト埋め込みモデルはありますか?'
documents = [
      'text: 更級日記は、平安時代中期に菅原孝標女によって書かれた回想録です。',
      'text: Sarashinaは、SB Intuitionsが開発した日本語大規模言語モデルです。これまでに7B, 13B, 70B, 8x70Bのモデルが公開されています。',
      'text: サラシナエンベディングは日本語言語モデルをベースにした日本語埋め込みモデルです。'
]
show_similar_words(embed, documents, query)

query: task: クエリを与えるので、与えられたWeb検索クエリに答える関連文章を検索してください。
query: Sarashinaのテキスト埋め込みモデルはありますか?
検索結果（類似度順）:
1. 類似度: 0.8777 - text: サラシナエンベディングは日本語言語モデルをベースにした日本語埋め込みモデルです。...
2. 類似度: 0.8648 - text: Sarashinaは、SB Intuitionsが開発した日本語大規模言語モデルです。こ...
3. 類似度: 0.7398 - text: 更級日記は、平安時代中期に菅原孝標女によって書かれた回想録です。...
